In [1]:
%load_ext autoreload
%autoreload 2

# Ragas evaluation
Test batch and ragas capability.

Uses this article as a model: https://towardsdatascience.com/visualize-your-rag-data-evaluate-your-retrieval-augmented-generation-system-with-ragas-fc2486308557

Ragas repository: https://github.com/explodinggradients/ragas/tree/main

In [2]:
import os, sys
import json
from pathlib import Path
import pickle

from ragas.testset import TestsetGenerator
from ragas import RunConfig
from dotenv import load_dotenv,find_dotenv
import chromadb
from chromadb import PersistentClient
from pinecone import Pinecone as pinecone_client, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_voyageai import VoyageAIEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_core.documents import Document
import pandas as pd
import random

from ragas import evaluate
from ragas.metrics import answer_correctness
from datasets import Dataset

from renumics import spotlight
from renumics.spotlight import Embedding
import pandas as pd

# from umap import UMAP
# import numpy as np

# Import local packages
sys.path.append('../src/aerospace_chatbot')
import queries
import eval
import admin
from data_processing import _stable_hash_meta, archive_db

# Set environment variables with .env
load_dotenv(find_dotenv(), override=True)

True

In [3]:
# Set secrets
secrets={}
sb={}

secrets['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
secrets['VOYAGE_API_KEY'] = os.getenv('VOYAGE_API_KEY')
secrets['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
secrets['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN')

# Create test (synthetic) dataset

Use chroma with standard RAG to generate synthetic dataset

## Connect to database

### ChromaDB

In [4]:
persistent_client = chromadb.PersistentClient(path=os.path.join(os.getenv('LOCAL_DB_PATH'),'chromadb'))   
collections=persistent_client.list_collections()
collections

[Collection(name=text-embedding-3-large-2merge-0),
 Collection(name=text-embedding-3-large-0merge-400),
 Collection(name=text-embedding-3-large-0merge-400-parent-child),
 Collection(name=text-embedding-3-large-2merge-0-queries)]

In [5]:
# Chroma _embedding_function isn't compatible like embedding objects. Index by embeddings used.
query_models=[OpenAIEmbeddings(model='text-embedding-3-large',openai_api_key=os.getenv('OPENAI_API_KEY')),
              OpenAIEmbeddings(model='text-embedding-3-large',openai_api_key=os.getenv('OPENAI_API_KEY')),
              OpenAIEmbeddings(model='text-embedding-3-large',openai_api_key=os.getenv('OPENAI_API_KEY')),
              OpenAIEmbeddings(model='text-embedding-3-large',openai_api_key=os.getenv('OPENAI_API_KEY')),
              OpenAIEmbeddings(model='text-embedding-3-large',openai_api_key=os.getenv('OPENAI_API_KEY'))]

In [6]:
# Export all collections to pickles to store them. Uncomment if desired, takes a while.
export=False

if export:
    for collection in collections:
        df_temp_chroma=archive_db('ChromaDB',collection.name,collection._embedding_function,export_pickle=True)

    df_temp_chroma.head(5)

### Pinecone

In [ ]:
pinecone_client = pinecone_client(api_key=os.getenv('PINECONE_API_KEY'))
indexes=pinecone_client.list_indexes()
indexes

In [ ]:
dbs=[{'index_name':'voyage-large-2-instruct-2merge-0',
     'query_model': VoyageAIEmbeddings(model='voyage-large-2-instruct', 
                                       voyage_api_key=os.getenv('VOYAGE_API_KEY'), truncation=False)},
     {'index_name':'voyage-large-2-instruct-0merge-400',
     'query_model': VoyageAIEmbeddings(model='voyage-large-2-instruct', 
                                       voyage_api_key=os.getenv('VOYAGE_API_KEY'), truncation=False)}]

In [ ]:
# Inspect the first db, save for synthetic test dataset
db=dbs[0]
index = pinecone_client.Index(db['index_name'])
ids=[]
for id in index.list():
    ids.extend(id)

docs=[]
chunk_size=200  # Tune to whatever doesn't error out, 200 won't for serverless
for i in range(0, len(ids), chunk_size):
    print(f"Fetching {i} to {i+chunk_size}")
    vector=index.fetch(ids[i:i+chunk_size])['vectors']
    vector_data = []
    for key, value in vector.items():
        vector_data.append(value)
    docs.extend(vector_data)

lcdocs_pinecone = []
for data in docs:
    data=data['metadata']
    lcdocs_pinecone.append(Document(page_content=data['page_content'],
                           metadata={'page':data['page'],'source':data['source']}))
    
print(len(lcdocs_pinecone))

In [ ]:
# Export all collections to pickles to store them
# for db in dbs:
#     df_temp_pinecone=archive_db('Pinecone',db['index_name'],db['query_model'],export_pickle=True)

## Create data for synthetic dataset

In [7]:
# Select database for determining synthetic dataset
idx=0   # Most reasonable baseline (text-embedding-3-large-2merge-0), top of the line embeddings, 2 page size good to genreate questions from.

docs_vectorstore=collections[idx]
query_model=query_models[idx]

In [8]:
# Inspect the first db, save for synthetic test dataset
all_docs = docs_vectorstore.get(include=["metadatas", "documents", "embeddings"])
lcdocs_chroma = [Document(page_content=doc, metadata=metadata) 
          for doc, metadata in zip(all_docs['documents'], all_docs['metadatas'])]

print(len(lcdocs_chroma))

2222


In [9]:
# Format docs into dataframe
all_docs = docs_vectorstore.get(include=["metadatas", "documents", "embeddings"])
df_docs = pd.DataFrame(
    {
        "id": [_stable_hash_meta(metadata) for metadata in all_docs["metadatas"]],
        "source": [metadata.get("source") for metadata in all_docs["metadatas"]],
        "page": [metadata.get("page", -1) for metadata in all_docs["metadatas"]],
        "document": all_docs["documents"],
        "embedding": all_docs["embeddings"],
    }
)

## Generate dataset

Good article on how models/embeddings are used in the `TestsetGenerator`: https://www.pondhouse-data.com/blog/evaluate-rag-performance-using-ragas

Sometimes you'll get a tricky threading error. Fully close vs studio, open a new window, restart the kernel, and it'll clear. It also appears to be related to versions newer than 0.1.6 for ragas. I'll stick with that for now until I find ways to test an upgrade.

In [ ]:
# Set generator inputs
generator_model="gpt-3.5-turbo-0125"
synthetic_generator_llm = ChatOpenAI(model=generator_model, tags=[generator_model])

critic_model='gpt-4o'
synthetic_critic_llm = ChatOpenAI(model=critic_model,tags=[critic_model])

# embedding_model='text-embedding-3-large'
# synthetic_embeddings = OpenAIEmbeddings(model=embedding_model,api_key=os.getenv('OPENAI_API_KEY'))
synthetic_embeddings=query_model

# Run parameters for testset generation
run_config=RunConfig(timeout=1000,
                max_retries=50,
                max_wait=1000,
                max_workers=1)

# Create generator
generator = TestsetGenerator.from_langchain(
    synthetic_generator_llm,
    synthetic_critic_llm,
    synthetic_embeddings,
    run_config=run_config
)

In [ ]:
# Input parameters
eval_size=100    # Number of samples to evaluate at a time. Intended to circumvent OpenAI API rate limits.
n_questions=10   # Number of questions to generate for each evaluation sample.
fname=os.path.join('output',f"testset_{docs_vectorstore.name}.csv")
lcdocs=lcdocs_chroma

In [ ]:
df_testset=eval.generate_testset(lcdocs,generator,eval_size,n_questions,fname,run_config)

# RAG Evaluation

This section will use the same base data as the synthetic test dataset but apply different RAG strategies:
* Different chunk sizes
* Embedding models
* LLMs
* Advanced RAG (parent-child, RAGatouille)

The database may not be the same as the synthetic test dataset but uses the same base data.

## Format dataset and database for RAG

In [23]:
# name=docs_vectorstore.name
testset_name='text-embedding-3-large-2merge-0'
fname=os.path.join('output',f"testset_{testset_name}.csv")

import_csv=True
if import_csv:
    df_testset = pd.read_csv(fname)

# temporarily reduce the quantity to evaluate the functionality
df_testset=df_testset.head(10)

In [24]:
df_questions = df_testset[['question', 'ground_truth']].copy()
df_questions['id'] = 'Question ' + df_questions.index.astype(str)
df_questions = df_questions[['id', 'question', 'ground_truth']]

In [25]:
# Load the cached RAG answers and source_documents ids from a file - or create an empty column
df_qa_template = eval.add_cached_column_from_file(
    df_questions, f"rag_response_cache_{testset_name}.txt", "question", "answer")

df_qa_template = eval.add_cached_column_from_file(
    df_qa_template, f"rag_response_cache_{testset_name}.txt", "question", "source_documents")

df_qa_template = eval.add_cached_column_from_file(
    df_qa_template, f"rag_response_cache_{testset_name}.txt", "question", "answer_by")

df_qa_template = eval.add_cached_column_from_file(
    df_qa_template, f"rag_response_cache_{testset_name}.txt", "question", "query_model")

From here, you have a blank dataframe to generate questions for an evaluate.

## Use RAG to generate responses

In [22]:
# Read setup data, determining the evaluation models and databases
json_file_path = "eval_models.json"
with open(json_file_path, "r") as json_file:
    setup_data = json.load(json_file)

# TODO add pinecone+voyage
# TODO add parent-child
# TODO add ragatouille

df_qa = pd.DataFrame()

for model in setup_data['eval_models']:
    print(model)
    
    index_type='ChromaDB'
    sb['index_type']=index_type
    index_name=testset_name
    sb['index_name']=index_name

    for key in model['query_model']:
        sb[key] = model['query_model'][key]
    query_model=admin.get_query_model(sb, secrets)
    for key in model['llm']:
        sb[key] = model['llm'][key]
    llm=admin.set_llm(sb, secrets)

    QA_model_params={'rag_type':'Standard',
                    'k':4,
                    'search_type':'similarity',
                    'local_db_path':os.getenv('LOCAL_DB_PATH')}
    
    df_qa_iter=eval.rag_responses(index_type, index_name, query_model, llm, QA_model_params, 
                                  df_qa_template, df_docs)
    df_qa = pd.concat([df_qa,df_qa_iter],ignore_index=True)

{'index_type': 'ChromaDB', 'index_name': 'text-embedding-3-large-2merge-0', 'query_model': {'query_model': 'OpenAI', 'embedding_name': 'text-embedding-3-large'}, 'llm': {'llm_source': 'OpenAI', 'llm_model': 'gpt-4o', 'model_options': {'temperature': 0.2, 'output_level': 1000}}, 'QA_model_params': {'rag_type': 'Standard', 'k': 4, 'search_type': 'similarity', 'local_db_path': '/Users/danmueller/Documents/GitHub/aerospace_chatbot/db'}}
Processing question 1/10
Processing question 2/10
Processing question 3/10
Processing question 4/10
Processing question 5/10
Processing question 6/10
Processing question 7/10
Processing question 8/10
Processing question 9/10
Processing question 10/10
{'index_type': 'ChromaDB', 'index_name': 'text-embedding-3-large-2merge-0', 'query_model': {'query_model': 'OpenAI', 'embedding_name': 'text-embedding-3-large'}, 'llm': {'llm_source': 'OpenAI', 'llm_model': 'gpt-4', 'model_options': {'temperature': 0.2, 'output_level': 1000}}, 'QA_model_params': {'rag_type': 'S

In [ ]:
# df_qa=eval.rag_responses(index_type, index_name, query_model, llm, QA_model_params, df_qa_template, df_docs)

## Ragas eval, visualize

In [ ]:
# Evaluate
df_qa = eval.eval_rag(index_name, df_qa)

In [ ]:
df_qa

In [ ]:
# Link from documents to questions, that used the document as source. Add UMAP column for visualization purposes.
df_visualize=eval.data_viz_prep(index_name,df_qa_eval,df_docs)

In [ ]:
# concat the df containing the questions and the df containing the documents
df = pd.read_parquet(f'df_{index_name}.parquet')

# show the dataframe with the question and answer in spotlight
spotlight.show(
    df,
    layout="https://spotlightpublic.blob.core.windows.net/docs-data/rag_demo/layout_rag_3.json",
    dtype={x: Embedding for x in df.keys() if "umap" in x},
)

##  UMAP visualization froms cluster of the questions, workaround: UMAP only on documents

# Pinecone

## Connect to database